# Preparing Spotify Data for Tableau
*Anne Bode*

In [2]:
import pandas as pd
import numpy as np
import requests

## Creating Streaming/Library Dataframe

In [3]:
# read your 1+ StreamingHistory files (depending on how extensive your streaming history is) into pandas dataframes
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('streamingHistory1.json')

# merge streaming dataframes
df_stream = pd.concat([df_stream0, df_stream1])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-09-12 00:01,Keri Hilson,Pretty Girl Rock,243920,Keri Hilson:Pretty Girl Rock
1,2020-09-12 00:05,Silk City,Electricity (with Dua Lipa),238173,Silk City:Electricity (with Dua Lipa)
2,2020-09-12 00:09,Chance the Rapper,Same Drugs,257775,Chance the Rapper:Same Drugs
3,2020-09-12 00:13,Frank Ocean,Thinkin Bout You,200746,Frank Ocean:Thinkin Bout You
4,2020-09-12 00:16,Zedd,The Middle,184732,Zedd:The Middle


In [4]:
# read your edited Library json file into a pandas dataframe
df_library = pd.read_json('YourLibrary1.json')

# add UniqueID column (same as above)
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']

# add column with track URI stripped of 'spotify:track:'
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]

df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Arctic Monkeys,Tranquility Base Hotel & Casino,Four Out Of Five,spotify:track:3nhzPKCm2yqGmgEhdAg19u,Arctic Monkeys:Four Out Of Five,3nhzPKCm2yqGmgEhdAg19u
1,Adele,19,First Love,spotify:track:1DHV4JhMnCkbLuf6Psg93N,Adele:First Love,1DHV4JhMnCkbLuf6Psg93N
2,Vampire Weekend,Modern Vampires of the City,Ya Hey,spotify:track:4eE6vZ2vOrceLq4xgz3VmG,Vampire Weekend:Ya Hey,4eE6vZ2vOrceLq4xgz3VmG
3,Haley Heynderickx,I Need to Start a Garden,Show You a Body,spotify:track:1Ax5RAq7lXMTF8A5PTqdl4,Haley Heynderickx:Show You a Body,1Ax5RAq7lXMTF8A5PTqdl4
4,Paris Jones,You're Invited (To the Assassination of Patric...,Summer,spotify:track:5rfvovWBd35hYqk6rWLIrr,Paris Jones:Summer,5rfvovWBd35hYqk6rWLIrr


In [5]:
# create final dict as a copy df_stream
df_tableau = df_stream.copy()

# add column checking if streamed song is in library
# not used in this project but could be helpful for cool visualizations
df_tableau['In Library'] = np.where(df_tableau['UniqueID'].isin(df_library['UniqueID'].tolist()),1,0)

# left join with df_library on UniqueID to bring in album and track_uri
df_tableau = pd.merge(df_tableau, df_library[['album','UniqueID','track_uri']],how='left',on=['UniqueID'])

df_tableau.head()

,endTime,artistName,trackName,msPlayed,UniqueID,In Library,album,track_uri
0,2020-09-12 00:01,Keri Hilson,Pretty Girl Rock,243920,Keri Hilson:Pretty Girl Rock,0,NaN,NaN
1,2020-09-12 00:05,Silk City,Electricity (with Dua Lipa),238173,Silk City:Electricity (with Dua Lipa),0,NaN,NaN
2,2020-09-12 00:09,Chance the Rapper,Same Drugs,257775,Chance the Rapper:Same Drugs,1,Coloring Book,6m9qPYXmhge2QhBLfFKnVF
3,2020-09-12 00:13,Frank Ocean,Thinkin Bout You,200746,Frank Ocean:Thinkin Bout You,1,channel ORANGE,7DfFc7a6Rwfi3YQMRbDMau
4,2020-09-12 00:16,Zedd,The Middle,184732,Zedd:The Middle,0,NaN,NaN


## Creating Genre Dataframe

In [6]:
# save your IDs from new project in Spotify Developer Dashboard
CLIENT_ID = '565c1e94bd89447eb3c564ceb55d3ab9'
CLIENT_SECRET = 'd41cc711389645f3b0fdf61695e70dca'

In [7]:
# generate access token

# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [8]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [9]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [10]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API,
# then use artist URI to pull genres associated with that artist
# store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [11]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,3nhzPKCm2yqGmgEhdAg19u,7Ln80lUS6He07XvHI8qqHH,"[garage rock, permanent wave, rock, sheffield ..."
1,1DHV4JhMnCkbLuf6Psg93N,4dpARuHxo51G3z768sgnrY,"[british soul, pop, pop soul, uk pop]"
2,4eE6vZ2vOrceLq4xgz3VmG,5BvJzeQpmsdsFp4HGUYUEx,"[baroque pop, indie rock, modern rock]"
3,1Ax5RAq7lXMTF8A5PTqdl4,73MDShZzdL4vUGMkmXOG6X,"[indie folk, indie pop, portland indie, small ..."
4,5rfvovWBd35hYqk6rWLIrr,1buXx0RSkvGYCCiAl03JIp,[]


In [13]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,3nhzPKCm2yqGmgEhdAg19u,7Ln80lUS6He07XvHI8qqHH,garage rock
0,3nhzPKCm2yqGmgEhdAg19u,7Ln80lUS6He07XvHI8qqHH,permanent wave
0,3nhzPKCm2yqGmgEhdAg19u,7Ln80lUS6He07XvHI8qqHH,rock
0,3nhzPKCm2yqGmgEhdAg19u,7Ln80lUS6He07XvHI8qqHH,sheffield indie
1,1DHV4JhMnCkbLuf6Psg93N,4dpARuHxo51G3z768sgnrY,british soul


In [15]:
# save df_tableau and df_genre_expanded as csv files that we can load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')
df_genre_expanded.to_csv('GenresExpandedTable.csv')

print('done')

done


In [17]:
import jovian
#jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "abode118/spotify-data-prep" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/abode118/spotify-data-prep


'https://jovian.ai/abode118/spotify-data-prep'